<a href="https://colab.research.google.com/github/sunnyna/TIL/blob/master/bond_analysis_report_crawling.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

drive 연결

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


# 채권 분석 리포트 크롤링

In [ ]:
import requests
from bs4 import BeautifulSoup

pdf_list = []
date_list = []
title_list = []

for i in range(23,127):
  url = 'https://finance.naver.com/research/debenture_list.nhn?'
  params = {
      'page' : i
  }

  resp = requests.get(url,params=params)
  soup = BeautifulSoup(resp.content, 'html.parser')

  td_tag = soup.findAll("td",{"class":"file"})
  date_tag = soup.findAll('td', {"style":"padding-left:5px"} )
  title_tag = soup.findAll('td', {"style":"padding-left:10px"})



  for t in td_tag:
    pdf_url=t.find('a').attrs['href']
    pdf_list.append(pdf_url)



  for date in date_tag:
    date_list.append(date.get_text())

  for title in title_tag:
    title_list.append(title.get_text())


갯수 확인 및 슬라이싱(08.04.01~17.12.27까지만)

In [ ]:
print(len(pdf_list))   
print(len(date_list))
print(len(title_list))


print(date_list[-10:])
print(date_list[:10])

슬라이싱 된 것 확인

In [ ]:

pdf_list_ = pdf_list[2:]
date_list_ = date_list[2:]
title_list_ = title_list[2:]

print(date_list_[-10:])
print(date_list_[:10])

In [ ]:
print(len(pdf_list_))
print(len(date_list_))
print(len(title_list_))


/ 가 경로오류가 생겨서 전처리해줌

In [ ]:
title_list__ = []
for t in title_list :
  title_list__.append(t.replace('/', ','))

title_list__

#해외크레딧/신흥국 국채 : 만기 짧은 미국 회사채에 ..' 
# /때문에 경로오류가생김

### pdf 파일로 저장
path 주의해서 확인하기

In [ ]:

for pdf,title,date in zip(pdf_list_, title_list__, date_list_ ):
  with open('/content/drive/My Drive/bond_analysis_report/{0}_{1}.pdf'.format(date,title), "wb") as file:
    response = requests.get(pdf)
    file.write(response.content)

파일갯수가 리스트 갯수와 일치하는 지 확인 (겹치는 제목은 덮어쓰기 때문에 제대로 나왔는지 확인해야했음)

In [ ]:
import os

path = "/content/drive/My Drive/bond_analysis_report"
file_list = os.listdir(path)

print ("file_list: {}".format(file_list))

len(file_list)

# pdf를 csv로 변환

In [2]:
!pip install tika
from tika import parser


  Created wheel for tika: filename=tika-1.24-cp36-none-any.whl size=32885 sha256=d4493c2e673606acf1e93c61fdfc474d97957a05f6a1770e26fbc394d69dd7ee
  Stored in directory: /root/.cache/pip/wheels/73/9c/f5/0b1b738442fc2a2862bef95b908b374f8e80215550fb2a8975
Successfully built tika


1. pdf파일을 txt로 변환하는 함수정의

In [ ]:
def convert_pdf_to_txt(file_name):
  rawText = parser.from_file('{}'.format(file_name))
  rawData = rawText['content']

  if rawData is not None:
    global rawData_ 
    rawData_ = rawData.replace('\n','') 


  return rawData_

'NoneType' object has no attribute 'replace' 오류수정  
None 일때 예외를 줘야됨

local variable 'rawData_' referenced before assignment 에러수정

지역범위에서 전역 변수를 사용하기 위해서 global 표현을 사용한다.



파일이름과 내용으로 이루어진 csv 파일 생성

In [ ]:
import csv
from pathlib import Path

folder = Path('/content/drive/My Drive/bond_analysis_report')
csv_file = Path('/content/drive/My Drive/bond_analysis_report_list.csv')

with csv_file.open('w', encoding='utf-8') as f:
    writer = csv.writer(f, csv.QUOTE_ALL)
    writer.writerow(['FileName', 'Text'])
    for pdf_file in folder.glob('*.pdf'):
        pdf_text = convert_pdf_to_txt(pdf_file)
        writer.writerow([pdf_file.name, pdf_text])


csv 파일을 df로 저장

In [ ]:
import pandas as pd

df = pd.read_csv('/content/drive/My Drive/bond_analysis_report_list.csv')
df

,FileName,Text
0,17.12.27_2018년 북한 리스크 대비하기.pdf,Microsoft Word - CrediVille_171227 트럼...
1,17.12.26_1월에는 무조건 GO.pdf,PowerPoint 프레젠테이션Fixed Income Analyst 김지만 02-3...
2,17.12.20_트럼프노믹스를 극복한 2017년 ᄉ...,2013년 0월 0일 2금융권 안심전환대출 수급영향 점검 Inve...
3,17.12.20_11월 채권시장 전망.pdf,Fixed Income 2017. 12. 20 11월 금통위 의...
4,17.12.18_2금융권 안심전환대출 수급영...,(Microsoft Word - \307\366\264\353\302\367_Cre...
...,...,...
3088,08.04.21_정부의 금리인하 요구에 한은의 부...,Microsoft Word - 920_0421_Fixed Income Weekly....
3089,08.04.14_FOMC 이후 원자재 가격 상승세 둔ᄒ...,Microsoft Word - Fixedincome0414.doc 4월 금통위...
3090,08.04.07_항상 열려는 있는 가능성.pdf,Microsoft Word - 0407.docFixed Income Weekly 2...
3091,08.04.07_총재의 변신은 무죄다?!.pdf,Microsoft Word - 0407.docFixed Income Weekly 2...


In [18]:
rawText = parser.from_file('/content/drive/My Drive/bond_analysis_report/08.04.07_총재의 변신은 무죄다?!.pdf')
rawList = rawText['content']

data =rawList.replace('\n','')

data

'Microsoft Word - 0407.docFixed Income Weekly 2008. 4. 7.  #918 Fixed Income Weekly 4월 7일돌아온 박스권 다만, 하단보다는 상단 테스트 가능성에 무게 글로벌 신용경색 완화 조짐과 주식의 반등, 인플레이션 및 금통위 부담 등에도 불구하고 금리가 잘 버티고 있다. 이는 물론, 대내외 펀더멘틀 및 궁극적인 통화정책 방향에대한 믿음이 강하기 때문일 것이다.  이러한 믿음이 견고한 상단을 만드는 반면, 추가적인 모멘텀 없이는 더 이상 하락하기쉽지않을 정도로 레벨이 낮아져 왔다는 현실이 역시 견고한 하단을 형성하고 있다. 결국 당분간 레인지 양상일 것으로 보이는데, 금통위 경계감 극대화시 trading buy 정도는 해 볼 만 하다는 생각이다.     대우증권 리서치센터 서철수 (02-768-3501)Cyberscs@bestez.com윤여삼 (02-768-4022)nersam@bestez.com미국의 금융 업종, 연준 덕택에 九死一生 연준 영역 밖인 회사채 신용위험은 추가 악화는 일단 멈춘 상태010020030040050060007.7 07.9 07.11 08.1 08.3은행업금융서비스업보험업(bp)1.52.02.53.03.54.007.1 07.3 07.5 07.7 07.9 07.11 08.1 08.30.60.81.01.21.41.6미 신용스프레드(회사채BAA-국채,좌)미 신용스프레드(회사채BAA-AAA,우)(%p)(%p)자료: Bloomberg 자료:  Bloomberg  2 ■ 최근 신용경색 완화 조짐!  So…what? 글로벌 신용경색 완화 조짐이 나타나면서 시장변수들이 움직이고 있다. 금융기관들의 CDS가 둔화되고 있고, 모기지 지수인 ABX나 회사채 신용스프레드 등도 추가 악화를 멈춘 상태이다[앞면의 그림들]. 달러가치의 급락이 멈칫해지는 가운데 상품가격의 조정양상이 진행중이며, flight to quality가 완화되면서 주가가 반등하고 있다[그림1,2]. 이러한 현상은, 4월 월보에서도 지적

In [25]:
import re 



no_eng_num_data = re.sub('([a-zA-Z0-9])','',data)
no_jamo_data = re.sub('[ㄱ-ㅎㅏ-ㅣ]+','',no_eng_num_data)
cleansing_data = re.sub('[-=+,#/\?:^$.@*\"※~&%ㆍ!』\\‘|\(\)\[\]\<\>`\'…》]','',no_jamo_data)


cleansing_data

'              월 일돌아온 박스권 다만 하단보다는 상단 테스트 가능성에 무게 글로벌 신용경색 완화 조짐과 주식의 반등 인플레이션 및 금통위 부담 등에도 불구하고 금리가 잘 버티고 있다 이는 물론 대내외 펀더멘틀 및 궁극적인 통화정책 방향에대한 믿음이 강하기 때문일 것이다  이러한 믿음이 견고한 상단을 만드는 반면 추가적인 모멘텀 없이는 더 이상 하락하기쉽지않을 정도로 레벨이 낮아져 왔다는 현실이 역시 견고한 하단을 형성하고 있다 결국 당분간 레인지 양상일 것으로 보이는데 금통위 경계감 극대화시   정도는 해 볼 만 하다는 생각이다     대우증권 리서치센터 서철수 윤여삼 미국의 금융 업종 연준 덕택에 九死一生 연준 영역 밖인 회사채 신용위험은 추가 악화는 일단 멈춘 상태    은행업금융서비스업보험업       미 신용스프레드회사채국채좌미 신용스프레드회사채우자료  자료     ■ 최근 신용경색 완화 조짐   글로벌 신용경색 완화 조짐이 나타나면서 시장변수들이 움직이고 있다 금융기관들의 가 둔화되고 있고 모기지 지수인 나 회사채 신용스프레드 등도 추가 악화를 멈춘 상태이다앞면의 그림들 달러가치의 급락이 멈칫해지는 가운데 상품가격의 조정양상이 진행중이며   가 완화되면서 주가가 반등하고 있다그림 이러한 현상은 월 월보에서도 지적했듯이 연준을 중심으로 미 정부와 의회 그리고 민간과 국부펀드 등이 신용경색 완화를 위한 다양한 노력들을 기울인 덕택일 것이다 특히 연준이 금리인하는 물론이고 금융기관에 직접 유동성을 공급할 뿐 아니라 부실처리까지 떠맡는 등 매우 적극적으로 대응하고 있는데 힘입은 바 크다고 할 것이다 더 이상 제의 베어스턴스를 만들지 않겠다는 연준의 강력한 의지와 실질적인 지원이 금융기관들 나아가 금융시장을 안심시키고 있는 것이다 이에 금융업종 중심의 안도랠리는 충분히 이해할 만한 구석이 있다는 생각이다 한편 버냉키는 의회 증언을 통해 이제 급한 불은 껐으니 향후 통화정책은 경기와 물가라는 전통적 잣대에 기반할 것임을 시사하였다긴급대폭적 인하 가